In this notebook, we will be using **Progressive Resizing** (technique made famous by Jeremy Howard). We will be using fastai library and pre-trained ResNet152. 

This is the training notebook. You can find the [inference notebook here](https://www.kaggle.com/ankursingh12/fastai-plant2021-starter-inference).

Lets get started . . . 

First, we will to import fastai.

In [ ]:
from fastai.vision.all import *
import gc

seed = 42
set_seed(seed, reproducible=True)

We are setting the seed for reproducibility. 

Next, we will initialize some path (& other) variables (for use throughout the notebook)

PS: I will be using my version for the dataset. I have resized all the images so that its much faster to load them into the RAM. You can find the dataset [here](https://www.kaggle.com/ankursingh12/resized-plant2021). 

In [ ]:
path = Path('../input/plant-pathology-2021-fgvc8')
data_path = Path('../input/resized-plant2021')

### Data

Enough prep-work! Lets read our data . . .

In [ ]:
df = pd.read_csv(path/'train.csv')
df.head()

hmm, just image names and their label. Looks simple ? Not so soon. The labels are space-delimited strings. Its a multi-label problem. 

We will use Fastai's datablock API to load our data. DataBlock API is simply amazing. Infinitely  flexibility and incredibly powerful. To use the datablock API, you need to define some functions.

In [ ]:
def get_x(x): return str(data_path/'img_sz_640') + os.path.sep + x['image']
def get_y(y): return y['labels']

def get_data(df, size=224, bs=64):
    datablock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(seed=seed),
                   get_x=get_x, get_y=get_y,
                   item_tfms = RandomResizedCrop(size+52),
                   batch_tfms=[*aug_transforms(mult=2.0,flip_vert=True, size=size), 
                               Normalize.from_stats(*imagenet_stats)])
    
    return datablock.dataloaders(df, bs=bs)

Don't worry a lot if the above code looks cryptic. You can read the [6th notebook (or chapter)](https://github.com/fastai/fastbook/blob/master/06_multicat.ipynb) in fastbook for details. It explains the topic in the most simplest way possible. And once you master datablock API, you will feel like a Ninja (trust me on this)!

You are amazing! Now lets create our dataloaders, & then take a look at some images.

In [ ]:
dls = get_data(df)
dls.show_batch(max_n=9)

Looks good to me, what do you think?

We are done with data, time for some training.

### Model

Fastai has an awesome class which puts everything together, called `cnn_learner`. Here we are using ResNet152. 

In [ ]:
f1score = F1Score(average='macro')
metrics = [accuracy, f1score]

We are using `accuracy_multi` and `f1score` metrics, because its a multi-label problem and the evaluation metric for the competition is *F1Score*.

Finally, lets train (technically, fine-tune 🤯) our model.

## Progressive Resizing

In Progressive resizing, we first train the model on smaller images and then progressively increase the size. 

**Philosophy:** Its much faster to train models on smaller image sizes. Also, the model sees more details as we increase the size. Hence, we can train the model much faster. 

We will start with 128x128 and gradually take the image size to 528x528

In [ ]:
## Size 128x128
dls = get_data(df, 128, 64)
learn = cnn_learner(dls, resnet152, metrics=metrics).to_fp16()
learn.fine_tune(1, 1e-3, wd=0.5)
learn.save('restnet_v1')
del learn
torch.cuda.empty_cache()
gc.collect()

In [ ]:
## Size 256x256
dls = get_data(df, 256, 32)
learn = cnn_learner(dls, resnet152, metrics=metrics)
learn.load('restnet_v1').to_fp16();
learn.freeze() 
learn.fine_tune(4, 1e-3, wd=0.5)
learn.save('restnet_v2')
del learn
torch.cuda.empty_cache()
gc.collect()

In [ ]:
## Size 448x448
dls = get_data(df, 448, 16)
learn = cnn_learner(dls, resnet152, metrics=metrics)
learn.load('restnet_v2').to_fp16();
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-5, 1e-4), wd=0.5)
learn.save('restnet_v3')
del learn
torch.cuda.empty_cache() 
gc.collect() 

In [ ]:
## Size 528x528
dls = get_data(df, 528, 16)
learn = cnn_learner(dls, resnet152, metrics=metrics)
learn.load('restnet_v3').to_fp16();
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-5, 1e-4), wd=0.5) 
learn.save('restnet_v4') 

Okay, done with training! Lets look at some predictions . . .

In [ ]:
learn.show_results()

Amazing! Lets export the model so that we can deploy it to production 😂. Just kidding, we will (only) use it for inference.

In [ ]:
learn.export(f'resnet152_12c.pkl')

Fastai is extremely flexible and powerful at the same time. This is just the baseline notebook. You can easily build on top of it. Here are some things that you can experiment with:

- Preprocess and Feature Engineering
- Data Augmentation and External Datasets
- Different Model Architectures
- Training Schedule, Optimizer, etc
- Postprocess

You can find the [inference notebook here](https://www.kaggle.com/ankursingh12/fastai-plant2021-starter-inference). Stay tuned for more!

Hope you had fun reading the notebook. Kindly consider **upvoting**.